In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# 5장 별형
* 샛별형
* 저녁별형
* 도지별형
* 유성형

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/A000250.csv')

In [ ]:
#샛별형 (하락추세 이후에 나타남)

'''
세 개의 캔들이 있음.
하락추세 - 이미 구현해둔 하락추세 사용
첫 번째 캔들 - 장대 음봉.(시가보다 종가가 크게 낮을 때 -> 종가가 시가보다 3% 이상 하락하는 경우로 정의)
두 번째 캔들 - 몸통이 앞의 음봉 몸통에 닿지 않는다.몸통이 작다(이미 구현해둔 small_body 정의 따름). 양/음봉상관없음
세 번째 캔들 - 첫 번째 캔들의 몸통 안으로 깊숙이 파고드는 양봉 -> 깊숙이 기준 0.5%로 정의
'''


def is_morning_star(df):
    downtrend = is_downtrend(df).shift(2).fillna(False)

    o = df['시가']
    h = df['고가']
    l = df['저가']
    c = df['종가']
    body = (o - c).abs()


    # 1일 전, 2일 전 값
    o1, o2 = o.shift(1), o.shift(2)
    c1, c2 = c.shift(1), c.shift(2)
    h1, h2 = h.shift(1), h.shift(2)
    l1, l2 = l.shift(1), l.shift(2)
    body_1, body_2 = body.shift(1), body.shift(2)

    #첫 번째 캔들
    first_bearish = c2 < o2
    first_long = ((o2 - c2) / o2) >= 0.03
    cond1 = (first_bearish & first_long).fillna(False)
    #두 번째 캔들
    second_small_body = body_1 < (h1 - l1) * 0.3
    #둘째 날 몸통의 상단이 첫째 날 몸통의 하단보다 밑에 있어야 함
    top_body_1 = pd.concat([o1, c1], axis=1).max(axis=1)
    bottom_body_2 = pd.concat([o2, c2], axis=1).min(axis=1)
    second_gap   = top_body_1 < bottom_body_2
    cond2 = (second_small_body & second_gap).fillna(False)

    #세 번째 캔들
    third_bullish = c > o
    third_penetration = (c - c2) / (o2 - c2) >= 0.5
    cond3 = (third_bullish & third_penetration).fillna(False)


    return (downtrend&cond1&cond2&cond3).fillna(False)


In [ ]:
df["샛별형"] = is_morning_star(df).astype(int)
print(df[df["샛별형"] == 1])
print(df)
#샛별형 없음

Empty DataFrame
Columns: [날짜, 시가, 고가, 저가, 종가, 거래량, 샛별형]
Index: []
            날짜      시가      고가      저가      종가      거래량  샛별형
0     20250414  157000  157000  150700  154000   233689    0
1     20250411  152200  162900  150500  157200   282408    0
2     20250410  156000  159300  151500  156600   326907    0
3     20250409  164200  169200  145100  146400   707729    0
4     20250408  185000  189300  157900  166800  1059177    0
...        ...     ...     ...     ...     ...      ...  ...
2452  20150420   12800   13300   12600   13000   627492    0
2453  20150417   13500   13850   12900   12900  2747398    0
2454  20150416   12250   12400   12100   12300   231163    0
2455  20150415   12350   12450   12000   12150   348865    0
2456  20150414   12800   13100   11850   12200   690688    0

[2457 rows x 7 columns]


/tmp/ipython-input-28-3819381928.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  downtrend = is_downtrend(df).shift(1).fillna(False)


In [ ]:
# 앞뒤내역 찾아보기
# 원본 마스크
mask = df['샛별형'] == 1

# 앞뒤 2행까지 확장된 마스크
mask_ext = (
    mask
    | mask.shift(1).fillna(False)
    | mask.shift(2).fillna(False)
    | mask.shift(-1).fillna(False)
    | mask.shift(-2).fillna(False)
)

# 출력
print(df[mask_ext])


Empty DataFrame
Columns: [날짜, 시가, 고가, 저가, 종가, 거래량, 샛별형]
Index: []


/tmp/ipython-input-30-812351262.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  | mask.shift(1).fillna(False)
/tmp/ipython-input-30-812351262.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  | mask.shift(2).fillna(False)
/tmp/ipython-input-30-812351262.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  | mask.shift(-1).fillna(Fals

In [ ]:
#저녁별형 (상승추세 이후에 나타남)

'''
세 개의 캔들이 있음.
상승추세 - 이미 구현해둔 상승추세 사용
첫 번째 캔들 - 장대 양봉.(시가보다 종가가 크게 높을 때 -> 종가가 시가보다 3% 이상 상승하는 경우로 정의)
두 번째 캔들 - 몸통이 앞의 양봉 몸통에 닿지 않는다.몸통이 작다(이미 구현해둔 small_body 사용) 양/음봉상관없음
세 번째 캔들 - 첫 번째 캔들의 몸통 안으로 깊숙이 파고드는 음봉 -> 깊숙이 기준 0.5%로 정의
'''

def evening_star(df):
    uptrend = is_uptrend(df).shift(2).fillna(False)

    o = df['시가']
    h = df['고가']
    l = df['저가']
    c = df['종가']
    body = (o - c).abs()


    # 1일 전, 2일 전 값
    o1, o2 = o.shift(1), o.shift(2)
    c1, c2 = c.shift(1), c.shift(2)
    h1, h2 = h.shift(1), h.shift(2)
    l1, l2 = l.shift(1), l.shift(2)
    body_1, body_2 = body.shift(1), body.shift(2)

    #첫 번째 캔들
    first_bullish = c2 > o2
    first_long = ((c2 - o2) / o2) >= 0.03
    cond1 = (first_bullish & first_long).fillna(False)

    #두 번째 캔들
    second_small_body = body_1 < (h1 - l1) * 0.3
    #둘째 날 몸통의 하단이 첫째 날 몸통의 상단보다 위에 있어야 함
    bottom_body_1 = pd.concat([o1, c1], axis=1).min(axis=1)
    top_body_2 = pd.concat([o2, c2], axis=1).max(axis=1)
    second_gap   = bottom_body_1 > top_body_2
    cond2 = (second_small_body & second_gap).fillna(False)

    #세 번째 캔들
    third_bullish = c < o
    third_penetration = (c2 - c) / (c2 - o2) >= 0.5
    cond3 = (third_bullish & third_penetration).fillna(False)


    return (uptrend&cond1&cond2&cond3).fillna(False)


In [ ]:
#십자샛별형 (하락추세 이후에 나타남)

'''
샛별형 중 두 번째 캔들이 도지인 경우 (전체 캔들 폭의 10% 이하인 경우로 정의)
'''

def is_morning_doji_star(df):
    downtrend = is_downtrend(df).shift(2).fillna(False)

    o = df['시가']
    h = df['고가']
    l = df['저가']
    c = df['종가']
    body = (o - c).abs()


    # 1일 전, 2일 전 값
    o1, o2 = o.shift(1), o.shift(2)
    c1, c2 = c.shift(1), c.shift(2)
    h1, h2 = h.shift(1), h.shift(2)
    l1, l2 = l.shift(1), l.shift(2)
    body_1, body_2 = body.shift(1), body.shift(2)

    #첫 번째 캔들
    first_bearish = c2 < o2
    first_long = ((o2 - c2) / o2) >= 0.03
    cond1 = (first_bearish & first_long).fillna(False)

    #두 번째 캔들
    second_doji_body = body_1 < (h1 - l1) * 0.1
    #둘째 날 몸통의 상단이 첫째 날 몸통의 하단보다 밑에 있어야 함
    top_body_1 = pd.concat([o1, c1], axis=1).max(axis=1)
    bottom_body_2 = pd.concat([o2, c2], axis=1).min(axis=1)
    second_gap   = top_body_1 < bottom_body_2
    cond2 = (second_doji_body & second_gap).fillna(False)

    #세 번째 캔들
    third_bullish = c > o
    third_penetration = (c - c2) / (o2 - c2) >= 0.5
    cond3 = (third_bullish & third_penetration).fillna(False)


    return (downtrend&cond1&cond2&cond3).fillna(False)


In [ ]:
#십자저녁별형 (상승추세 이후에 나타남)

'''
저녁별형 중 두 번째 캔들이 도지인 경우 (전체 캔들 폭의 10% 이하인 경우로 정의)

'''

def is_evening_doji_star(df):
    uptrend = is_uptrend(df).shift(2).fillna(False)

    o = df['시가']
    h = df['고가']
    l = df['저가']
    c = df['종가']
    body = (o - c).abs()

    # 1일 전, 2일 전 값
    o1, o2 = o.shift(1), o.shift(2)
    c1, c2 = c.shift(1), c.shift(2)
    h1, h2 = h.shift(1), h.shift(2)
    l1, l2 = l.shift(1), l.shift(2)
    body_1, body_2 = body.shift(1), body.shift(2)

    #첫 번째 캔들
    first_bullish = c2 > o2
    first_long = ((c2 - o2) / o2) >= 0.03
    cond1 = (first_bullish & first_long).fillna(False)

    #두 번째 캔들
    second_doji_body = body_1 < (h1 - l1) * 0.1
    #둘째 날 몸통의 하단이 첫째 날 몸통의 상단보다 위에 있어야 함
    bottom_body_1 = pd.concat([o1, c1], axis=1).min(axis=1)
    top_body_2 = pd.concat([o2, c2], axis=1).max(axis=1)
    second_gap   = bottom_body_1 > top_body_2
    cond2 = (second_doji_body & second_gap).fillna(False)

    #세 번째 캔들
    third_bullish = c < o
    third_penetration = (c2 - c) / (c2 - o2) >= 0.5
    cond3 = (third_bullish & third_penetration).fillna(False)


    return (uptrend&cond1&cond2&cond3).fillna(False)


In [ ]:
#유성형(하락반전)

'''
주가상승 마지막에 출현
하나의 캔들
1. 몸통이 작다, 몸통의 색깔은 중요하지 않다
2. 윗그림자가 길다
3. 아랫그림자가 거의 없다


'''

def is_shooting_star(df):
    uptrend = is_uptrend(df).shift(1).fillna(False)

    o = df['시가']
    h = df['고가']
    l = df['저가']
    c = df['종가']
    body = (o - c).abs()

    # 작은몸통
    small_body = body < (h - l) * 0.3

    # 긴 윗그림자
    higher_shadow = h - pd.concat([o, c], axis=1).max(axis=1)
    long_upper_tail = higher_shadow > 2 * body

    # 거의 없는 아랫그림자
    lower_shadow = pd.concat([o, c], axis=1).min(axis=1) - l
    thin_lower = lower_shadow < 0.1 * body


    return (uptrend&small_body&long_upper_tail&thin_lower).fillna(False)


In [ ]:
#역망치형(상승반전)

'''
하락세 마지막에 출현
하나의 캔들
1. 몸통이 작다, 몸통의 색깔은 중요하지 않다
2. 윗그림자가 길다
3. 아랫그림자가 거의 없다
'''

def is_inverted_hammer(df):
    downtrend = is_downtrend(df).shift(1).fillna(False)

    o = df['시가']
    h = df['고가']
    l = df['저가']
    c = df['종가']
    body = (o - c).abs()

    # 작은몸통
    small_body = body < (h - l) * 0.3

    # 긴 윗그림자
    higher_shadow = h - pd.concat([o, c], axis=1).max(axis=1)
    long_upper_tail = higher_shadow > 2 * body

    # 거의 없는 아랫그림자
    lower_shadow = pd.concat([o, c], axis=1).min(axis=1) - l
    thin_lower = lower_shadow < 0.1 * body


    return (downtrend&small_body&long_upper_tail&thin_lower).fillna(False)
